<center><h1>A data-driven approach for the assessment of the thermal stratification of reservoirs based on readily available data.</h1> 

<center><h4><span style='color:royalblue'> María Castrillo </span>, Fernando Aguilar, Daniel García
<center> (castrillo@ifca.unican.es)

In [ ]:
#Import libraries and pack
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import scipy
import sklearn
import mlxtend
import tensorflow as tf
import shap
import scikeras

In [ ]:
from scikeras.wrappers import KerasRegressor, KerasClassifier

In [ ]:
from tensorflow import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Dropout, SimpleRNN

In [ ]:
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold, cross_val_score, cross_validate
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_absolute_error

In [ ]:
plt.rcParams["font.family"] = "serif"

In [ ]:
tf.keras.utils.set_random_seed(42)

In [ ]:
#Read the data
data = pd.read_csv("dailyElVal.csv", sep=",")
data["Date"] = pd.to_datetime(data["Date"])

In [ ]:
data.info()

In [ ]:
data.describe()

## Outliers detection

In [ ]:
#After visual inspection we eliminate observations with RadMax > 800 and RadAvg < 180, which are obvious outliers in this time series.
data = data[data["RadMax"] < 800]
data = data[data["RadAvg"] < 180]

In [ ]:
data.describe()

## Violin plots

In [ ]:
cols = ["AirTempMax", "AirTempMin", "AirTempAvg", "Wind", "RadMax", "RadAvg", "Rain", "Level", "Flow", "Flow"]

dark_gray = "#000000"
light_gray = "#eeeeee"

fig, axs = plt.subplots(2, 5, figsize=(12, 6), layout='constrained')

for i, col in enumerate(cols): 
    ax = axs.flat[i]
    sns.violinplot(y=data[col], ax=ax,  color="lightsalmon")
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_color(dark_gray)
    ax.spines["bottom"].set_color(dark_gray)
    ax.set_facecolor('xkcd:white')
    
    ax.get_yaxis().tick_left()
    ax.get_xaxis().tick_bottom()
    
    ax.tick_params(axis="both", which="major", labelsize=12, colors=dark_gray)
    ax.tick_params(axis="both", which="minor", labelsize=10, colors=dark_gray)
    ax.xaxis.label.set_color(dark_gray)
    ax.set_ylabel(cols[i], fontsize=14)
    
fig.delaxes(ax)

## Data shifts

In [ ]:
data[["AirTempAvg1", "Wind1", "AirTempMin1", "AirTempMax1", "RadMax1", "RadAvg1", "Rain1", "Level1", "Flow1"]]=data[["AirTempAvg", "Wind", "AirTempMin", "AirTempMax", "RadMax", "RadAvg", "Rain", "Level", "Flow"]].shift(1, axis = 0)
data[["AirTempAvg2", "Wind2", "AirTempMin2", "AirTempMax2", "RadMax2", "RadAvg2", "Rain2", "Level2", "Flow2"]]=data[["AirTempAvg", "Wind", "AirTempMin", "AirTempMax", "RadMax", "RadAvg", "Rain", "Level", "Flow"]].shift(2, axis = 0)
data[["AirTempAvg3", "Wind3", "AirTempMin3", "AirTempMax3", "RadMax3", "RadAvg3", "Rain3", "Level3", "Flow3"]]=data[["AirTempAvg", "Wind", "AirTempMin", "AirTempMax", "RadMax", "RadAvg", "Rain", "Level", "Flow"]].shift(3, axis = 0)
data[["AirTempAvg4", "Wind4", "AirTempMin4", "AirTempMax4", "RadMax4", "RadAvg4", "Rain4", "Level4", "Flow4"]]=data[["AirTempAvg", "Wind", "AirTempMin", "AirTempMax", "RadMax", "RadAvg", "Rain", "Level", "Flow"]].shift(4, axis = 0)
data[["AirTempAvg5", "Wind5", "AirTempMin5", "AirTempMax5", "RadMax5", "RadAvg5", "Rain5", "Level5", "Flow5"]]=data[["AirTempAvg", "Wind", "AirTempMin", "AirTempMax", "RadMax", "RadAvg", "Rain", "Level", "Flow"]].shift(5, axis = 0)

# Modelling

## Unsupervised detection of the thermocline

In [ ]:
#data["Date"]  = pd.to_datetime(data["Date"], format = "%Y-%M-%d").dt.normalize()
#Separate train and test fractions
data2 = data.drop(columns = ["Thermocline_depth", "Thermocline_temp"])
train = data2[data2["Date"] <= "2021-12-31"].dropna()
test = data2[data2["Date"] >= "2022-01-01"].dropna()

In [ ]:
#Define inputs and target variable
X = train[["AirTempAvg", "Wind", "AirTempMin", "AirTempMax", "RadMax", "RadAvg", "Rain", "Level", "Flow"]]
y = train["Thermocline"]

In [ ]:
#Scale the input
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
#Define and train the model
kmeans = KMeans(n_clusters= 2, 
            random_state=42)

kmeans.fit(X_scaled)

#Predict the clusters
kmeans_pred = kmeans.predict(X_scaled)

In [ ]:
kmeans_pred = pd.DataFrame(kmeans_pred).set_index(y.index)

In [ ]:
#Obtain clusters centers
kmeans.cluster_centers_

In [ ]:
train = train.merge(kmeans_pred, left_index=True, right_index=True)
train = train.rename(columns = {'0_x':"kmeans"})

In [ ]:
fig = plt.figure(figsize=(12,6))

plt.plot(train.Date, (train.Gradient), linewidth=0.5, color='black')
plt.plot(train.Date[train.kmeans==0], (train.Gradient[train.kmeans==0]), 
         linestyle=" ", marker="D", markersize=3, color='coral', label="Cluster 0")
plt.plot(train.Date[train.kmeans==1], (train.Gradient[train.kmeans==1]), 
         linestyle=" ", marker="o", markersize=4,  color='slateblue', label="Cluster 1")

plt.xlabel('Time', fontsize = 14)
plt.ylabel('Thermal gradient 1 m around Z', fontsize = 14)
plt.xticks(())
plt.yticks(np.arange(0,15,1))
plt.hlines(y=0.3, xmin=17500, xmax=19000, colors='red', linestyles='-', lw=1, label="0.3 \u2103 $\mathregular{m^{-1}}$")
plt.legend()

dark_gray = "#000000"
light_gray = "#eeeeee"

ax = fig.axes[0]
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_color(dark_gray)
ax.spines["bottom"].set_color(dark_gray)

ax.yaxis.grid(True, color=light_gray)
ax.xaxis.grid(True, color=light_gray)

import matplotlib.dates as mdates
ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 7)))

## Classification neural network

In [ ]:
X_train = train[[
                 "AirTempAvg", "Wind", "AirTempMin", "AirTempMax", "RadMax", "RadAvg", "Rain", "Level", "Flow",
                 "AirTempAvg1", "Wind1", "AirTempMin1", "AirTempMax1", "RadMax1", "RadAvg1", "Rain1", "Level1", "Flow1",
                 "AirTempAvg2", "Wind2", "AirTempMin2", "AirTempMax2", "RadMax2", "RadAvg2", "Rain2", "Level2", "Flow2",
                 "AirTempAvg3", "Wind3", "AirTempMin3", "AirTempMax3", "RadMax3", "RadAvg3", "Rain3", "Level3", "Flow3",
                 "AirTempAvg4", "Wind4", "AirTempMin4", "AirTempMax4", "RadMax4", "RadAvg4", "Rain4", "Level4", "Flow4",
                 "AirTempAvg5", "Wind5", "AirTempMin5", "AirTempMax5", "RadMax5", "RadAvg5", "Rain5", "Level5", "Flow5",
                ]]
y_train = train["Thermocline"]
X_test = test[[        
                 "AirTempAvg", "Wind", "AirTempMin", "AirTempMax", "RadMax", "RadAvg", "Rain",  "Level", "Flow",
                 "AirTempAvg1", "Wind1", "AirTempMin1", "AirTempMax1", "RadMax1", "RadAvg1", "Rain1", "Level1", "Flow1", 
                 "AirTempAvg2", "Wind2", "AirTempMin2", "AirTempMax2", "RadMax2", "RadAvg2", "Rain2", "Level2", "Flow2",
                 "AirTempAvg3", "Wind3", "AirTempMin3", "AirTempMax3", "RadMax3", "RadAvg3", "Rain3", "Level3", "Flow3",
                 "AirTempAvg4", "Wind4", "AirTempMin4", "AirTempMax4", "RadMax4", "RadAvg4", "Rain4", "Level4", "Flow4",
                 "AirTempAvg5", "Wind5", "AirTempMin5", "AirTempMax5", "RadMax5", "RadAvg5", "Rain5", "Level5", "Flow5",
              ]]
y_test = test["Thermocline"]

In [ ]:
#Scaling
scaler = StandardScaler()

X_train[X_train.columns] = scaler.fit_transform(X_train)
X_test[X_test.columns] = scaler.transform(X_test)

In [ ]:
#Define the architecture of the NN
NN_class = Sequential([
    Dense(32, input_shape=(len(X_train.columns),)),
    Activation('relu'),
     Dropout(0.2),
    Dense(16),
    Activation("relu"),
    Dropout(0.2),
    Dense(1), 
    Activation('sigmoid'),
])

### Cassification neural network with cross-validation

In [ ]:
#Compile the NN
metric = tf.keras.metrics.BinaryAccuracy(
    name="binary_accuracy", dtype=None, threshold=0.5
)

opt = keras.optimizers.Adam(learning_rate=0.0001)

NN_class.compile(loss = "binary_crossentropy",
                optimizer = opt, 
                metrics=metric)

In [ ]:
model = KerasClassifier(NN_class, batch_size=32, verbose=1, epochs=500)

In [ ]:
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc'}

In [ ]:
kfold = KFold(n_splits=4, shuffle=False)
results = cross_validate(model, X_train, y_train, cv=kfold, scoring=scoring)

In [ ]:
#Cross-validation results
results

In [ ]:
#Averaged results
print("Accuracy: " + str(np.mean(results["test_acc"])))
print("Recall: " + str(np.mean(results["test_recall"])))
print("AUC: " + str(np.mean(results["test_auc"])))

### Cassification neural network without cross-validation

In [ ]:
#Compile the NN
metric = tf.keras.metrics.BinaryAccuracy(
    name="binary_accuracy", dtype=None, threshold=0.5
)

opt = keras.optimizers.Adam(learning_rate=0.0001)

NN_class.compile(loss = "binary_crossentropy",
                optimizer = opt, 
                metrics=metric)

In [ ]:
#Train the NN with validation split
log_class = NN_class.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.25)

In [ ]:
#Plot the train accuracy and validation accuracy
plt.plot(log_class.history['binary_accuracy'])
plt.plot(log_class.history['val_binary_accuracy'])
plt.legend(['accuracy','validation accuracy'])

In [ ]:
NN_class.fit(X_train, y_train, epochs=200, batch_size=32)

In [ ]:
scores = NN_class.evaluate(X_train, y_train)
print("%s: %.2f%%" % (NN_class.metrics_names[1], scores[1]*100))

In [ ]:
scores = NN_class.evaluate(X_test, y_test)
print("%s: %.2f%%" % (NN_class.metrics_names[1], scores[1]*100))

In [ ]:
y_pred_train = NN_class.predict(X_train)
y_pred = NN_class.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

fig, ax = plt.subplots(figsize=(8,6))
cm_train = confusion_matrix(y_train, y_pred_train.round())  #target and predictions
display = ConfusionMatrixDisplay(cm_train)
ax.set(title='Train Confusion Matrix')
display.plot(ax=ax)

fig, ax = plt.subplots(figsize=(8,6))
cm_test = confusion_matrix(y_test, y_pred.round())
display = ConfusionMatrixDisplay(cm_test)
ax.set(title='Test Confusion Matrix')
display.plot(ax=ax)

In [ ]:
from sklearn.metrics import RocCurveDisplay

RocCurveDisplay.from_predictions(y_train, y_pred_train)
RocCurveDisplay.from_predictions(y_test, y_pred)

In [ ]:
from sklearn.metrics import recall_score

recall_train = recall_score(y_train, y_pred_train.round())
recall_test = recall_score(y_test, y_pred.round())

print(recall_train)
print(recall_test)

In [ ]:
y_pred_train = pd.Series(np.array(y_pred_train.round(0).reshape(-1-1)))
y_pred_train.index = y_train.index

In [ ]:
y_pred = pd.Series(np.array(y_pred.round(0).reshape(-1-1)))
y_pred.index = y_test.index

In [ ]:
yy_train = pd.concat((data["Date"], y_train), axis=1).set_index("Date", drop=True)
yy_pred_train = pd.concat((data["Date"], y_pred_train), axis=1).set_index("Date", drop=True)
yy_test = pd.concat((data["Date"], y_test), axis=1).set_index("Date", drop=True)
yy_pred = pd.concat((data["Date"], y_pred), axis=1).set_index("Date", drop=True)

In [ ]:
fig = plt.figure(figsize=(12,3))

plt.plot(yy_train+0.05, linewidth=0.0, marker = "o", markersize = 7, color="darkslateblue")
plt.plot(yy_test+0.05, linewidth=0.0, marker = "o", markersize = 7,label="Real data", color="darkslateblue")
plt.plot(yy_pred_train-0.05, linewidth=0.0, marker = "o", markersize = 7,  color="lightsalmon")
plt.plot(yy_pred-0.05, linewidth=0.0, marker = "o", markersize = 7, label="Predicted data", color="lightsalmon")

plt.axvline(x = 18994, color = 'black', ls='--', lw=1)

plt.xlabel("Date", fontsize = 14)
plt.ylabel("Thermocline presence", fontsize = 14)
plt.legend(loc="center left", facecolor="white")

dark_gray = "#000000"
light_gray = "#eeeeee"

ax = fig.axes[0]
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_color(dark_gray)
ax.spines["bottom"].set_color(dark_gray)

ax.set_facecolor('xkcd:white')

ax.get_yaxis().tick_left()
ax.get_xaxis().tick_bottom()

ax.tick_params(axis="both", which="major", labelsize=10, colors=dark_gray)
ax.tick_params(axis="both", which="minor", labelsize=8, colors=dark_gray)
ax.xaxis.label.set_color(dark_gray)
ax.yaxis.label.set_color(dark_gray)

ax.set_yticks((0,1))

ax.yaxis.grid(True, color=light_gray, lw=20)
ax.xaxis.grid(True, color=light_gray)

ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 7)))

## Regression neural network

In [ ]:
train = data[data["Date"] <= "2021-12-31"].dropna()
test = data[data["Date"] >= "2022-01-01"].dropna()

In [ ]:
X_train = train[[
                "AirTempAvg", "Wind", "AirTempMin", "AirTempMax", "RadMax", "RadAvg", "Rain", "Level", "Flow",
                "AirTempAvg1", "Wind1", "AirTempMin1", "AirTempMax1", "RadMax1", "RadAvg1", "Rain1", "Level1", "Flow1",
                "AirTempAvg2", "Wind2", "AirTempMin2", "AirTempMax2", "RadMax2", "RadAvg2", "Rain2", "Level2", "Flow2",
                "AirTempAvg3", "Wind3", "AirTempMin3", "AirTempMax3", "RadMax3", "RadAvg3", "Rain3", "Level3", "Flow3",
                "AirTempAvg4", "Wind4", "AirTempMin4", "AirTempMax4", "RadMax4", "RadAvg4", "Rain4", "Level4", "Flow4",
                "AirTempAvg5", "Wind5", "AirTempMin5", "AirTempMax5", "RadMax5", "RadAvg5", "Rain5", "Level5", "Flow5",
                ]]
y_train = train["Thermocline_depth"]
X_test = test[[             
                "AirTempAvg", "Wind", "AirTempMin", "AirTempMax", "RadMax", "RadAvg", "Rain",  "Level", "Flow",
                "AirTempAvg1", "Wind1", "AirTempMin1", "AirTempMax1", "RadMax1", "RadAvg1", "Rain1", "Level1", "Flow1", 
                "AirTempAvg2", "Wind2", "AirTempMin2", "AirTempMax2", "RadMax2", "RadAvg2", "Rain2", "Level2", "Flow2",
                "AirTempAvg3", "Wind3", "AirTempMin3", "AirTempMax3", "RadMax3", "RadAvg3", "Rain3", "Level3", "Flow3",
                "AirTempAvg4", "Wind4", "AirTempMin4", "AirTempMax4", "RadMax4", "RadAvg4", "Rain4", "Level4", "Flow4",
                "AirTempAvg5", "Wind5", "AirTempMin5", "AirTempMax5", "RadMax5", "RadAvg5", "Rain5", "Level5", "Flow5",
              ]]
y_test = test["Thermocline_depth"]

In [ ]:
# Scaling
scaler = StandardScaler()

X_train[X_train.columns] = scaler.fit_transform(X_train)
X_test[X_test.columns] = scaler.transform(X_test)

In [ ]:
#Define the architecture of the NN
NN_regr = Sequential([
    Dense(32, input_shape=(len(X_train.columns),)),
    Activation('relu'),
    Dropout(0.4),
    Dense(16),
    Activation('relu'),
    Dropout(0.2),
    Dense(8),
    Activation('relu'),
    Dense(1), 
])

### Regression neural network with cross-validation

In [ ]:
#Compile
opt = keras.optimizers.Adam(learning_rate=0.001)

NN_regr.compile(loss= "mae",
                optimizer=opt, 
                metrics= "mae")

In [ ]:
model = KerasRegressor(NN_regr, batch_size=32, verbose=1, epochs=500)

In [ ]:
kfold = KFold(n_splits=4, shuffle=False)
results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='neg_mean_absolute_error')

In [ ]:
#Cross-validation results
results

In [ ]:
#Averaged results
np.mean(abs(results))

### Regression neural network without cross-validation

In [ ]:
#Compile
opt = keras.optimizers.Adam(learning_rate=0.001)

NN_regr.compile(loss= "mae",
                optimizer=opt, 
                metrics= "mae")

In [ ]:
#Train the NN with validation split
log_rgr = NN_regr.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.25)

In [ ]:
#Plot the train accuracy and validation accuracy
plt.plot(log_rgr.history['mae'])
plt.plot(log_rgr.history['val_mae'])
plt.legend(['mae','validation mae'])

In [ ]:
NN_regr.fit(X_train, y_train, epochs=500, batch_size=32)

In [ ]:
scores = NN_regr.evaluate(X_train, y_train)
print("%s: %.2f" % (NN_regr.metrics_names[1], scores[1]))

In [ ]:
scores = NN_regr.evaluate(X_test, y_test)
print("%s: %.2f" % (NN_regr.metrics_names[1], scores[1]))

In [ ]:
y_pred_train = NN_regr.predict(X_train)
y_pred = NN_regr.predict(X_test)

In [ ]:
y_pred = pd.Series(np.array(y_pred.reshape(-1-1)))
y_pred.index = y_test.index

In [ ]:
y_pred_train = pd.Series(np.array(y_pred_train.reshape(-1-1)))
y_pred_train.index = y_train.index

In [ ]:
MAE_train = np.mean(np.abs((y_train-y_pred_train)))
MAE_test = np.mean(np.abs((y_test-y_pred)))
P50_train = np.quantile(np.abs(y_train-y_pred_train), 0.50)
P50_test = np.quantile(np.abs(y_test-y_pred), 0.50)
print("P50_train: " + str(P50_train))
print("P50_test: " + str(P50_test))
print("MAE_train percentile: " + str((np.sum(np.abs(y_train-y_pred_train) < MAE_train) / len(y_train-y_pred_train) * 100).round(2)))
print("MAE_test percentile: " + str((np.sum(np.abs(y_test-y_pred) < MAE_test) / len(y_test-y_pred) * 100).round(2)))

In [ ]:
print("MAE_test percentile: " + str((np.sum(np.abs(y_test-y_pred) < MAE_test) / len(y_test-y_pred) * 100).round(2)))

In [ ]:
dates_train = data["Date"].iloc[y_train.index]
dates_test = data["Date"].iloc[y_test.index]

In [ ]:
y_pred_high_error = y_pred[abs(y_pred-y_test) > scores[1]]
y_pred_low_error = y_pred[abs(y_pred-y_test) <= scores[1]]

In [ ]:
yy_train = pd.concat((data["Date"], y_train), axis=1).set_index("Date", drop=True)
yy_pred_train = pd.concat((data["Date"], y_pred_train), axis=1).set_index("Date", drop=True)
yy_test = pd.concat((data["Date"], y_test), axis=1).set_index("Date", drop=True)
yy_pred_high_error = pd.concat((data["Date"], y_pred_high_error), axis=1).set_index("Date", drop=True)
yy_pred_low_error = pd.concat((data["Date"], y_pred_low_error), axis=1).set_index("Date", drop=True)

In [ ]:
fig = plt.figure(figsize=(12,6))

plt.plot(data["Thermocline_depth"], linewidth=0, marker = "o", markersize = 5, label="Real data", color="darkslateblue")
plt.plot(y_pred_train, linewidth=0, marker = "^", markersize = 6, label="Predicted data (train)", color="#f57d6b", markeredgecolor="darksalmon")
plt.plot(y_pred_high_error, linewidth=0, marker = "^", markersize = 6, label="Predicted data (test-high)", color="#ffb14e", markeredgecolor="orange")
plt.plot(y_pred_low_error, linewidth=0, marker = "^", markersize = 6, label="Predicted data (test-low)", color="#ffff52", markeredgecolor="gold")

plt.xlabel("Date", fontsize = 14)
plt.ylabel("Z (m)", fontsize = 14)
plt.legend(loc="upper left", facecolor="white")

dark_gray = "#000000"
light_gray = "#eeeeee"

ax = fig.axes[0]
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_color(dark_gray)
ax.spines["bottom"].set_color(dark_gray)

ax.set_facecolor('xkcd:white')

ax.get_yaxis().tick_left()
ax.get_xaxis().tick_bottom()

ax.tick_params(axis="both", which="major", labelsize=10, colors=dark_gray)
ax.tick_params(axis="both", which="minor", labelsize=8, colors=dark_gray)
ax.xaxis.label.set_color(dark_gray)
ax.yaxis.label.set_color(dark_gray)

ax.yaxis.grid(True, color=light_gray)
ax.xaxis.grid(True, color=light_gray)

ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 7)))


In [ ]:
fig = plt.figure(figsize=(12,6))
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.plot(data["Thermocline_depth"], linewidth=0, marker = "o", markersize = 3, label="Real data (train)", color="darkslateblue")
ax2.plot(data["Level"], linewidth=0, marker = "o", markersize = 3, label="Level", color="red")

## SHAP values

In [ ]:
# Fits the explainer
explainer = shap.Explainer(NN_regr.predict, X_train)

# Calculates the SHAP values 
shap_values = explainer(X_test)

In [ ]:
# Summarize the effects of features
shap.plots.beeswarm(shap_values, color=plt.get_cmap("plasma"))

In [ ]:
shap.plots.bar(shap_values)

In [ ]:
abs(shap_values.values).mean(axis=0)